In [1]:
import datetime as dt
import numpy as np
import pandas as pd

In [2]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [3]:
from flask import Flask, jsonify

In [4]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [5]:
Base = automap_base()

In [6]:
Base.prepare(engine, reflect=True)

In [7]:
measurement = Base.classes.measurement
station = Base.classes.station

In [8]:
session = Session(engine)

In [9]:
app = Flask(__name__)

In [10]:
#import app
print("example __name__ = %s", __name__)

if __name__ == "__main__":
    print("example is being run directly.")
else:
    print("example is being imported")

example __name__ = %s __main__
example is being run directly.


In [11]:
@app.route("/")
def welcome():
	return(
        f"Welcome to the Climate Analysis API!<br/>"
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/temp/start/end<br/>"
	)

In [12]:
@app.route("/api/v1.0/precipitation")
def precipitation():
    prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
    precipitation = session.query(Measurement.date, Measurement.prcp).\
            filter(Measurement.date >= prev_year).all()
    precip = {date: prcp for date, prcp in precipitation}
    return jsonify(precip)

In [13]:
@app.route("/api/v1.0/stations")
def stations():
    results = session.query(Station.station).all()
    stations = list(np.ravel(results))
    return jsonify(stations)

In [17]:
@app.route("/api/v1.0/tobs")
def temp_monthly():
    prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
    results = session.query(Measurement.tobs).\
        filter(Measurement.station = 'USC00519281').\
        filter(Measurement.date >= prev_year).all()
    temps = list(np.ravel(results))
    return jsonify(temps)

SyntaxError: invalid syntax (<ipython-input-17-7a29862114da>, line 4)

In [ ]:
@app.route("/api/v1.0/temp/<start>")
@app.route("/api/v1.0/temp/<start>/<end>")

def stats():

    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs),              func.max(Measurement.tobs)]

    if not end:
        results = session.query(*sel).\
            filter(Measurement.date >= start).\
            filter(Measurement.date <= end).all()
        temps = list(np.ravel(results))
        return jsonify(temps)
     
    results = session.query(*sel).\
     filter(Measurement.date >= start).\
     filter(Measurement.date <= end).all()
     temps = list(np.ravel(results))
    return jsonify(temps)

In [ ]:
if __name__=='__main__':
    app.run()